# RSA
_____

**References:**
* [[GCAC](https://toc.cryptobook.us/)] A Graduate Course in Applied Cryptography, Dan Boneh and Victor Shoup
* [[CINTA](https://shoup.net/ntb/)] A Computational Introduction to Number Theory and Algebra, Victor Shoup
* [[HAC](https://cacr.uwaterloo.ca/hac/)] Handbook of Applied Cryptography, Alfred J. Menezes, Paul C. van Oorschot and Scott A. Vanstone
* [[ITMC](https://www.cs.umd.edu/~jkatz/imc.html)] Introduction to Modern Cryptography, Jonathan Katz and Yehuda Lindell (3rd Edition)
* [Elementary number theory in sage](http://doc.sagemath.org/html/en/constructions/number_theory.html)

**Dependencies:**

This notebook uses [sage math](https://www.sagemath.org/) as kernel

In [1]:
import sage.misc.banner # sage math version info 
rchars = "┘─│┐┌└"
smallbanner = sage.misc.banner.banner_text(full=True)
for c in rchars:
    smallbanner = smallbanner.replace(c,"") # remove ascii art box
print(smallbanner)
#banner() # full banner with box


 SageMath version 9.5, Release Date: 2022-01-30                     
 Using Python 3.10.6. Type "help()" for help.                       



## RSA - Rivest Shamir Adelman
* **R**ivest **S**hamir **A**delman (RSA)
    + Adi Shamir, Ron Rivest, Leonard Adelman (all MIT at that time)
<p style="text-align:center">
<img src="./img/RSA.jpg" alt="number systems" width="600">
</p>
<p style="text-align:center;font-size:10px">
<a href="https://en.wikipedia.org/wiki/Prime_number#/media/File:Primes-vs-composites.svg">(image source)</a>
</p>
* Algorithm first published 1977
* Key length of $> 2048$ bits still considered secure [[ref](https://www.keylength.com/)]

## Trapdoor function
A *trapdoor function* is a triple of efficient algorithms $(G,F,F^{-1})$ where:
* $G()$ is a key generation algorithm that outputs $(pk,sk)$
* $F(pk,\cdot)$ the $pk$ defines a mapping function $X \to Y$
* $F^{-1}(sk,\cdot)$ defines a function $Y \to X$ that *inverts* $F(pk,\cdot)$

It holds that $\forall(pk,sk)$ that are the output of $G()$:
$$
\forall x \in X: F^{-1}(sk, F(pk,x)) = x 
$$
*Note:* If both sets are equal ($X = Y$) then then it is a *trapdoor permutation*. A secure trapdoor function/permutation is one-way without the trapdoor $sk$

## Trapdoor function
![RSA_trapdoor](./figures/RSA_trapdoor.png)

## RSA - Trapdoor permutation

The RSA *trapdoor permutation* is a triple of efficient algorithms $\langle G,F,F^{-1} \rangle$
* $G()$ key generation function:
  + Choose two random primes $p,q$ of at least $ 1024 $ bit each and compute the $ 2048 $ bit *modulus* $n =pq $
  + Choose integers $e,d$ such that $e\cdot d = 1 \pmod{\varphi(n)}$
      - There are some commonly used values for $ e $ such as $ e = 3 $ or $ e = 2^{16} + 1 = 65537$
  + Output $pk = (n,e)$ and $sk=(n,d)$
      - The variabel $ e $ is called the *encryption exponent* and $ d $ is called the *decryption exponent*. 
* $F(pk,x) = x^e \pmod{\mathbb{Z}_n}$
    + This trapdoor permutation defines a mapping between all invertable elements $\mathbb{Z}_n^* \to \mathbb{Z}_n^*$
* $F^{-1}(sk,y) = y^d \pmod{\mathbb{Z}_n}$
    + This is the inverse trapdoor permutation 

To show that $F^{-1}(sk, F(pk,x)) = x $ we use the fact that there exists some $ k $ such that $ ed = k \cdot \varphi(n) + 1 $ since $ e \cdot d = 1 \pmod{\varphi(n)} $. 

If $x$ is *relatively prime* to $n$, i.e., $\gcd(x,n) = 1$, then $x^{\varphi(n)} \equiv 1 \pmod n$ by *Euler's theorem*.

$$
y^d = (x^e)^d = x^{ed} = x^{k\cdot \varphi(n)+1} = \left( x^{\varphi(n)}\right)^k \cdot x = (1)^k \cdot x = x \pmod n
$$

There is also a proof using *Fermat's little theorem* which does not require that $\gcd(x,n)=1$, although it is hard to find an $x$ (message) for which this is not the case as there are only $1/p + 1/q - 1/(pq)$ numbers have this property.
[(source)](https://en.wikipedia.org/wiki/RSA_(cryptosystem)#Proofs_of_correctness)

## RSA - Security 
* Computing $\varphi(n)$ without knowing the factorization of $ n $ is the difficulty of computing $ d $
* This is refered to as the *RSA problem*, which can be solved by factoring $ n $ and computing $\varphi(n)$.
* Knowing $\varphi(n)$, the multiplicative inverse of $ e $ can be calculated efficiently:
$$
\begin{align*}
\varphi(n) &= (p-1)(q-1)\\
e\cdot e^{-1} &=1 \pmod{\varphi(n)}\\
d&:= e^{-1} 
\end{align*}
$$

## Textbook/raw RSA implementation (insecure)
<a id='raw'></a>
This *textbook* implementation of RSA is insecure, but we will use it to demonstrate a couple of security issues 
which have to be addressed when RSA should be used in a secure way.

```python
def rsa_gen(bits=2^1024,_p=None,_q=None,e=None,verbose=False):
	proof = (bits <= 2^1024) # turn off time consuming proof for large values 
	if _p and _q and _p in Primes() and _q in Primes():
		p = _p
		q = _q
	else:
		p = random_prime(bits, proof=proof)
		q = random_prime(bits, proof=proof)
  if p==q: return None	
  n = p * q
  phi_n = (p-1) * (q-1)
  if not e:
    while True:
        e = ZZ.random_element(1,phi_n)
        if gcd(e,phi_n) == 1: break
  d = inverse_mod(e,phi_n)
  if verbose: print(f"n = {n}\ne = {e}\nd = {d}\np = {p}\nq = {q}")
  return n,e,d,p,q 
```

```python
def rsa_enc(n,e,m):
	return pow(m,e,n)

def rsa_dec(n,d,c):
	return pow(c,d,n)
```

```python
def str_to_num(s,n):
    """ Function to encode a string as number sequence of their 8 bit ASCII codes """
	s = str(s)
	if len(s) > floor(log(n,256)):
			print("String larger than what can be handled in one rsa invocation")
			return None
	num = 0
	for i in range(len(s)):
		num += ord( s[i] )*256^i  # the ^i inverts 
	return num 	

def num_to_str(num):
    """ Function to decode a number into 8 bit sequences of ASCII characters """
	num = Integer(num) 
	v = [] 
	while num != 0: 
    	v.append(chr( num % 256 )) 
		num //= 256 # this replaces num by floor(num/256) 
	return ''.join(v)
```

<p style="text-align:center">
<img src="./img/rsa-perl.jpg" alt="number systems" width="450">
</p>
<p style="text-align:center;font-size:10px">
<a href="https://www.wired.com/wp-content/uploads/blogs/wiredenterprise/wp-content/uploads/2013/01/rsa-perl.jpg">RSA printed on a T-shirt during the first "crypto wars" in the 90s</a>
</p>

In [38]:
load('./src/rsa.sage')

In [37]:
!cat ./src/rsa.sage

#!/usr/bin/sage
#
# Simple sage script for RSA 
# 
# See sources:
# * https://www.math.ucdavis.edu/~anne/FQ2010/Number_Theory_RSA.html
# * https://www.uam.es/otros/openmat/software/files/rsa.sage
# * https://www.hyperelliptic.org/tanja/vortraege/facthacks-29C3.pdf

def rsa_gen(bits=2^1024,_p=None,_q=None,e=None,verbose=False):
	proof = (bits <= 2^1024) # turn off time consuming proof for large values 
	if _p and _q and _p in Primes() and _q in Primes():
		p = _p
		q = _q
	else:
		p = random_prime(bits, proof=proof)
		q = random_prime(bits, proof=proof)
	assert p!=q,"p and q must be distinct numbers!"
	n = p * q
	phi_n = (p-1) * (q-1)

	if not e: 
		while True:
				e = ZZ.random_element(1,phi_n)
				if gcd(e,phi_n) == 1: break

	d = inverse_mod(e,phi_n)
	
	if verbose: print(f"n = {n}\ne = {e}\nd = {d}\np = {p}\nq = {q}")
	return n,e,d,p,q 

def rsa_enc(n,e,m):
	return pow(m,e,n)

def rsa_dec(n,d,c):
	return pow(c,d,n)

def str_to_num(s,n):
	s = str(s)
	if len(s) > floor(log(n,256)):
			

### Check properties 

In [39]:
n,e,d,p,q = rsa_gen(bits=2^100,e=None,verbose=True)

n = 355976772106829268621600816711847149676690353471135641076853
e = 179769921186679219769808393315989015768769534121126583859697
d = 58431222555913008042476852133249811347510347288162823550921
p = 862498832840930945935888344079
q = 412727250811806492026541947707


In [23]:
n

588828197890170242005998523619419274940260660598198069167549

In [24]:
phi_n=(p-1)*(q-1)
phi_n 

588828197890170242005998523617689415036808015850180029804764

In [25]:
phi_n < n

True

In [26]:
n - phi_n

1729859903452644748018039362785

In [27]:
hpRR= RealField(128) # custom high precision real numbers 
hpRR(phi_n/n) 

0.99999999999999999999999999999706219928

In [28]:
mod(e*d,phi_n) == 1

True

In [29]:
k=5
mod(k*phi_n+1,phi_n) == 1

True

In [30]:
x=10
assert gcd(x,n) == 1,"x and n are not relatively prime"
pow(x,phi_n,n) == 1 

True

### Raw RSA - Exampel (insecure)
* Lets pick some primes $p=2$ and $q=11$ and compute $n = pq = 22$
* Compute $\varphi(n) = (p-1)(q-1) = 10$
* Find some $e$ such that:
  + It is *relatively prime* to $\varphi(n)$, i.e., $\gcd(e,\varphi(n))=1$
  + There is a $d$ such that $e\cdot d = 1 \pmod \varphi(n)$
* For our example set $ e = 3 $ and $ d = 7 $, i.e., $ 3 \cdot 7 = 1 \pmod \varphi(n)$
* Publish $ pk = (n,e) = (22,3) $ and store $ sk = (n,d) = (22,7) $
* **Encryption:** of message $ m = 8 $
  + $ c = m^e \pmod n = 8^3 \pmod{22} = 512 \pmod{22} = 6 $
* **Decryption:** of ciphertext $ c = 6 $
  + $ m = c^d \pmod n = 6^7 \pmod{22} = 279936 \pmod{22} = 8 $

In [40]:
n,e,d,_,_ = rsa_gen(bits=None,_p=2,_q=11,e=3,verbose=True)

n = 22
e = 3
d = 7
p = 2
q = 11


In [41]:
rsa_enc(22,3,8)

6

In [42]:
rsa_dec(22,7,6)

8

### Raw RSA - Parameters (p,q) too small
If $p$ and $q$ are to small $n$ can be factorized:

In [50]:
n,e,d,_,_ = rsa_gen(bits=None,_p=2,_q=11,e=3,verbose=True) # Alice 

n = 22
e = 3
d = 7
p = 2
q = 11


In [55]:
n=22; e=3 # Attacker uses pk of Alice

In [56]:
factor(n) # factors n into its prime factors

2 * 11

In [57]:
phi_n=(2-1)*(11-1); phi_n # computes phi with known factors 

10

In [58]:
d=inverse_mod(e,phi_n);d # compute secret key by inverting e 

7

#### Factorization examples

In [76]:
n,e,d,p,q = rsa_gen(bits=2^32,e=None,verbose=True) # 2 x 2**32 bit primes 

n = 7726023075241080047
e = 3129271511086424969
d = 4895170541792722169
p = 3344967287
q = 2309745481


In [71]:
%time factor(n)

CPU times: user 885 µs, sys: 0 ns, total: 885 µs
Wall time: 915 µs


667245871 * 811987919

In [72]:
%time euler_phi(n)

CPU times: user 7.41 ms, sys: 0 ns, total: 7.41 ms
Wall time: 7.4 ms


541795584775398660

#### Factorization examples

In [73]:
%time factor(random_prime(2**32)*random_prime(2**32)) # 64 bit n

CPU times: user 1.39 ms, sys: 175 µs, total: 1.57 ms
Wall time: 1.42 ms


555933967 * 3233872093

In [74]:
%time factor(random_prime(2**64)*random_prime(2**64)) # 128 bit n 

CPU times: user 31.9 ms, sys: 0 ns, total: 31.9 ms
Wall time: 31.5 ms


13391792971847536679 * 15730516463031225431

In [75]:
%time factor(random_prime(2**96)*random_prime(2**96)) # 192 bit n

CPU times: user 2.47 s, sys: 0 ns, total: 2.47 s
Wall time: 2.43 s


47419285361993254485888859339 * 55372916850435414411345734849

In [77]:
%time factor(random_prime(2**96)*random_prime(2**96)) # 256 bit n

CPU times: user 2.47 s, sys: 2.2 ms, total: 2.47 s
Wall time: 2.45 s


22442486277199865063599153657 * 77948547969620138234416312829

#### Factorization examples
<p style="text-align:center">
<img src="./figures/factoring.png" alt="number systems" width="600">
</p>
<p style="text-align:center;font-size:10px">
<a href="https://i.stack.imgur.com/VSwml.png">(image source)</a>
</p>

### $p$ and $q$ must be approximately the same size
If one prime factor is too small then factoring is easier [cf.](https://en.wikipedia.org/wiki/Lenstra_elliptic_curve_factorization)

In [78]:
%time factor(random_prime(2**1024)*random_prime(2**32))

CPU times: user 9.47 s, sys: 32.9 ms, total: 9.51 s
Wall time: 9.51 s


698567819 * 94692169340483632461114630164328963698879505170556566972045795393051833317801747401489930226118272217098274686134608761496913501709622812987818880419682606181187147160481334590216466952325163147397121582578615062622481469881353566201891954541233814301021239975979330964635810437129266455167917642618890216211

In [79]:
%time factor(random_prime(2**2048)*random_prime(2**32))

CPU times: user 44.3 s, sys: 114 ms, total: 44.4 s
Wall time: 44.3 s


1332360613 * 4327997323590140857125228350507204300016645189765317199650543153076737614908393372613197309658718570515179368036062403284687117286053738393636717962032986295052903594080327091986679599097411692972353834546689362563652960322673011449148921752638730702191811317064618933330504862220435379072711133459671011284417392743391218569475755646316614168584425798762801342998499152594150868004355420925694838836948714027350795197293310316013247105817242813473812325977809603048030131316237590865698050573208772907952628533625733100242575911188490427224562909402068006293688007472338307201187463769097084210135153611260657065503

### $p$ and $q$ must be approximately the same size
Increasing the number of prime factors is also not a good idea:

In [84]:
bits=32
8*bits

256

In [151]:
%time factor(random_prime(2**bits)*random_prime(2**bits)*random_prime(2**bits)\
             *random_prime(2**bits)*random_prime(2**bits)*random_prime(2**bits)\
             *random_prime(2**bits)*random_prime(2**bits))

CPU times: user 125 ms, sys: 0 ns, total: 125 ms
Wall time: 125 ms


497551679 * 1213179103 * 1877973649 * 1910631941 * 2321971139 * 2447232373 * 2827747159 * 3397759657

**Raw RSA:** (now assume large $p$ and $q$, other problems?)
* Lets pick some primes $p$ and $q$ and compute $n = pq $
* Compute $\varphi(n) = (p-1)(q-1) $
* Find some $e$ such that:
  + It is *relatively prime* to $\varphi(n)$, i.e., $\gcd(e,\varphi(n))=1$
  + There is a $d$ such that $e\cdot d = 1 \pmod \varphi(n)$
* Publish $ pk = (n,e) $ and store $ sk = (n,d) $
* **Encryption:** of message $ m = 8 $
  + $ c = m^e \pmod n  $
* **Decryption:** of ciphertext $ c = 6 $
  + $ m = c^d \pmod n  $

### Raw RSA - Data patterns visible
Raw RSA is deterministic encryption, the same plain text leads to the same ciphertext. 
Moreover, since the public key $pk$ of Alice is public, an attacker can create a dictionary of common values. 
```bash
Alice  <----- E(n,e,"all good") ----- Bob
Alice  <----- E(n,e,"error") -------- Bob
```

In [153]:
n,e,d,p,q = rsa_gen(bits=2^1024,e=65537,verbose=False)

In [154]:
num = str_to_num("error",n)

In [166]:
c = rsa_enc(n,e,num); str(c)[0:50] + "..." # always the same c under (n,e) 

'20498062452290032520263199700981021515763327797300...'

In [157]:
m_num = rsa_dec(n,d,c); m = num_to_str(m_num); m

'error'

### Raw RSA - Integrity not ensured
The integrity of the ciphertext is not ensured. Therefore, the ciphertext is malleable. 

In [97]:
n,e,d,_,_ = rsa_gen(bits=2**1024,e=65537) 

In [105]:
c = rsa_enc(n,e,8); c # small message from Bob e.g., sensor value

1056716605771145727559769161954840204128587371239515379942595789152543316784121164088665635483131910030299900595685860325614557281966260116383558106250725435028411023123877103580823382117213950343822479598350828479916249589260070857465202042192757762583437015436464055605391007835101234860329304643347260406263625493110394071177738976073779767574544425155221164924109498215613577950138038637451591657002337358499297536387999173284816654808324074343060419991573110704420008947996546490400306405245266626087160356272911298484365632701803460439732274243392099141490231384718628140937911973655622010371813614367092424130

In [112]:
m = rsa_dec(n,d,c); m # Alice decrypts c

8

In [ ]:
m = rsa_dec(n,d,c); m # Alice decrypts c

In [113]:
c1 = c*c # MitM attacker just multiplies the value with itself

In [114]:
m = rsa_dec(n,d,c1); m # Alice decrypts c1

64

In [115]:
m==8*8

True

In [116]:
c2 = c*rsa_enc(n,e,2) # MitM attacker can calc. factor using pk

In [118]:
m = rsa_dec(n,d,c2); m # Alice decrypts c2

16

In [119]:
m==8*2

True

### Raw RSA - $e^{th}$ root attack on small $m$
If $m$ is small $m < n^{1/e}$ and $ e $ is small, e.g., $e = 3$ and $ m^e < n $, then
you can simply calculate the eth root ($\sqrt[e] m$) within the set of integers and decrypt the ciphertext

In [142]:
n,e,d,_,_ = rsa_gen(bits=2**1025,e=3,verbose=True)

n = 8500974778571891654568169108350813086773181226084685198811763567613870576796014635662611012535852060657504599807897323587526362757688091920247045770863705693122457662275701214407780998450912357565868474451845853279269438909378650105783309351640250876518813690548272618560118743331225536195928310719944983487479818344323408684621336820511934820034825318904389528150822115716750759686759702116192793255612489467652357751539435286606342331206012996779893767524126783045550461189715440337728996968380780081639350950440063184668177636408491595273784510082971752467270394198509711161576690448379388132238164874993359392431
e = 3
d = 566731651904792776971211273890054205784878748405645679920784237840924705119734309044174067502390137377166973320526488239168424183845872794683136384724247046208163844151713414293852066563394157171057898296789723551951295927291910007052220623442683391767920912703218174570674582888748369079728554047996332232485591422400160098398303572939734297840955354271538434646009557

In [143]:
num = str_to_num("A",n); num; hex(num)

'0x41'

In [148]:
c = rsa_enc(n,3,65); c # Bob encrypts message m=65 for Alice

274625

In [149]:
274625.nth_root(3) # Attacker can compute plaintext 

65

## Other RSA gotchas 
Over the years a lot of attacks against RSA have been [studied](https://crypto.stanford.edu/~dabo/papers/RSA-survey.pdf). This shows that getting it right is indeed hard, but still it has not been compleatly broken. 

* *Coppersmith attack*
    + Low encryption exponent $ e $ and small message $ m $
    + Or partial knowledge of secret key
* *Wiener's attack*
    + Low decryption exponent $ d $
* *Hastad's broadcast attack*
    + Same message to different public key (moduli) with same (small) $ e $
* *Meet in the middle attack*
    + If small (e.g., 128 bi) non padded value is transferred that is the product of two (e.g., 64 bit) values
* *Common factor attacks across multiple keys*
    + Same prime factors in different moduli
    + [Mining your p's and q's](https://factorable.net/weakkeys12.extended.pdf)
* *Side channels*
    + e.g., timing, power consumption, ...
* ...

## RSA but how?
In general: If alternative are available better use them.
RSA is just a trapdoor permutation and should never be used directly as encryption system (textbook RSA). 
It has to be embedded/transformed before usage, e.g., OAEP, PKCS1v2.0, or ISO 18033-2.

ISO 18033-2 standard roughly:
* $(E_s,D_s)$: Symmetric encryption/ decryption system which provides authenticated encryption
* $H(): \mathbb{N}_n \to K $: Secure hash function that maps elements of $\mathbb{Z}_n $ to secret keys from a symmetric encryption/decryption system.
* $G():$ Generate RSA parameter $pk = (n,e)$, $sk = (n,d)$
* $E(pk,m)$: choose random $x$ in $\mathbb{Z}_n$
  + $y \gets x^e \pmod{\mathbb{Z}_n}$, and $ k \gets H(x)$, also *padding* is applied
  + output and send $(y,E_s(k,m))$
* $D(sk,(y,c))$: $m = D_s(H(y^d \pmod{\mathbb{Z}_n}),c)$


# EOF